In [21]:
'''
<<<design>>>

1. input 형태 조정 *****
 1.-(1) MFCC 파일을 x, y로 구분하여 리스트에 저장
 (x : 컬럼 차원 리스트, y : 클래스 라벨(1차원 리스트))
2. input을 받아서 svm kernel function을 바꿔서 돌리기
 2-(2) 클래스 여러 개인 부분 조정
 2-(3) kernel function 변경

'''

from sklearn import svm
from sklearn.datasets import make_blobs
from random import *

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import librosa
import librosa.display
from matplotlib.pyplot import specgram
import os
import IPython.display as ipd
import random


In [106]:

#.wav 파일을 받아와서 MFCC로 변환
# mean 을 통하여 정규화를 해서 집어넣는다고 함.
def wav2mfcc(path):
    mfcc_x = []
    label_y = []
    for subdir, dirs, files in os.walk(path):
        for file in files:
            try:
                X, sample_rate = librosa.load(os.path.join(subdir,file), res_type = "kaiser_fast", duration = 2.5, sr = 22050*2, offset = 0.5)
                mfcc = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc = 40).T,axis=0)
                y = int(file[7:8])-1
                mfcc_x.append(mfcc)
                label_y.append(y)
            except ValueError:
                continue

    return mfcc_x, label_y


#list를 판다스 df로 보여주기
def print_df(mylist):
    df = pd.DataFrame(data = mylist)
    print(df)


def train_test_split(x, y, train_size):
    
    # 두 리스트를 합쳐서, 셔플하여 조정
    # 이후 train size 만큼 나눠서 분배
    
    tmp = [[a,b] for a, b in zip(x, y)]
    random.shuffle(tmp)
    x2 = [n[0] for n in tmp]
    y2 = [n[1] for n in tmp]
    index = int(len(y) * train_size)
    x_train = []
    x_test = []
    x_train = x2[:index]
    x_test = x2[index:]
    y_train = y2[:index]
    y_test = y2[index:]
    '''
        x_train = x[:index]
    x_test = x[index:]
    y_train = y[:index]
    y_test = y[index:]

    '''


    return x_train, x_test, y_train, y_test

def accuracy_score(error,total):
    return error / total





In [115]:


'''
-------------------------------------------------
                      main
-------------------------------------------------
'''

#input
'''
listData = readCSVtoList('output.csv') #전역변수랑 로컬변수랑 이름 같아도 되나?
x, x_name = get_x(listData)
y = get_y(listData)
'''

# 모든 데이터 집어넣음
x, y = wav2mfcc("C:\\Users\\JW\\JPNotebook\\O2DD\\ravdess-video\\")
x2, y2 = wav2mfcc("C:\\Users\\JW\\JPNotebook\\O2DD\\ravdess-emotional-speech-audio\\")
x3, y3 = wav2mfcc("C:\\Users\\JW\\JPNotebook\\O2DD\\ravdess-emotional-song-audio\\")







In [122]:
x_data = np.array(x)
y_labels= np.array(y)
x2_data = np.array(x2)
y2_labels= np.array(y2)
x3_data = np.array(x3)
y3_labels= np.array(y3)

data = np.r_[x_data, x2_data, x3_data]
labels = np.r_[y_labels, y2_labels, y3_labels]

data.shape


#data split
x_train, x_test, y_train, y_test = train_test_split(data, labels, 0.70)

#print(x_train)
print_df(x_train)
print_df(y_train)
print_df(x_test)
print_df(y_test)


              0           1          2          3          4          5   \
0    -485.166474   77.100276 -53.131326  12.786502 -10.281740 -12.346160   
1    -516.793461   58.939177 -11.463089   5.428594  -6.699784   7.535803   
2    -539.790985   74.557256  -9.768580   9.223550   0.304534   5.943291   
3    -557.094951   90.361897  -3.148035  19.784330  -8.178239   3.433803   
4    -564.682985  121.460753 -10.495018  18.660840   6.961421  16.517341   
...          ...         ...        ...        ...        ...        ...   
3427 -605.279944  101.181173   0.000021  16.274840   1.661630  16.281983   
3428 -556.081996   74.953394 -25.908829   5.917469   3.268165  -1.057469   
3429 -459.838489   59.822244 -30.759273  10.118008 -18.427122  -4.580034   
3430 -419.590801   65.296982  -6.246971  25.175383  -8.408934  13.609742   
3431 -602.373659   91.608361 -10.854101  10.595830  -8.113948   5.040261   

             6          7          8          9   ...         30         31  \
0    -29

In [123]:

#training session
clf = svm.SVC(kernel='linear')
clf.fit(x_train, y_train)

#test session
y_pred = clf.predict(x_test)
print('총 테스트 개수: %d, 오류개수:%d' %(len(y_test), (y_test != y_pred).sum()))
#print('정확도: %.2f' % accuracy_score(y_test,y_pred))

# 비고
'''
트레이닝 데이터와 테스트 데이터의 비율은 생각보다는 크게 영향이 없었다.

'''


총 테스트 개수: 1472, 오류개수:562


'\n트레이닝 데이터와 테스트 데이터의 비율은 생각보다는 크게 영향이 없었다.\n\n'